In [1]:
!pip install kaggle

In [1]:
!pip install kaggle
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/ml_sample

/content/drive/MyDrive/ml_sample


In [3]:
!ls


emotiondetector.h5    face-expression-recognition-dataset.zip  facialexpression.json
emotiondetector.json  facialexpression.h5		       kaggle.json


In [4]:
!cp '/content/drive/MyDrive/kaggle.json' '/content/drive/MyDrive/kaggle.json'

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d jonathanoheix/face-expression-recognition-dataset
# Unzip the dataset



cp: -r not specified; omitting directory '/content/drive/MyDrive/kaggle.json'
401 - Unauthorized - Unauthenticated


In [5]:

!ls

emotiondetector.h5    face-expression-recognition-dataset.zip  facialexpression.json
emotiondetector.json  facialexpression.h5		       kaggle.json


In [ ]:
!unzip -q face-expression-recognition-dataset.zip -d /content/yt


replace /content/yt/images/images/train/angry/0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
from IPython.display import Image, display

# Specify the file path
image_path = "/content/yt/images/images/train/angry/1.jpg"

# Display the image
display(Image(filename=image_path))

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import os
import pandas as pd
import numpy as np


In [ ]:
TRAIN_DIR = "/content/yt/images/train"
TEST_DIR = "/content/yt/images/validation"

In [ ]:
import pandas as pd
import glob
import os

def create_dataframe(dir):
    image_paths = []
    labels = []

    for label in os.listdir(dir):
        for imagename in glob.glob(os.path.join(dir, label, '*.*')):
            image_paths.append(imagename)
            labels.append(label)
        print(label, "completed")

    data = {'Image_Path': image_paths, 'Label': labels}
    df = pd.DataFrame(data)

    return df


In [ ]:
import os
import pandas as pd

image_paths, labels = create_dataframe(TRAIN_DIR)

train = pd.DataFrame()
train['image'] = image_paths
train['label'] = labels


In [ ]:
print(train)

In [ ]:
image_paths, labels = create_dataframe(TEST_DIR)

test = pd.DataFrame()
test['image'] = image_paths
test['label'] = labels

In [ ]:
print(test)

In [ ]:
import cv2
import numpy as np
import glob
from tqdm import tqdm

def extract_features(images):
    features = []

    for image_path in tqdm(images):
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (48, 48))  # Resize the image to the desired shape
        img = img.reshape((48, 48, 1))  # Add channel dimension
        features.append(img)

    features = np.array(features)
    return features


In [ ]:
train_features = extract_features(train['image'])
print(train_features.shape)


In [ ]:
test_features = extract_features(test['image'])
print(test_features.shape)


In [ ]:
x_train = train_features/255.0
x_test = test_features/255.0

In [20]:
!pip install -U scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 38.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [21]:
from sklearn.preprocessing import LabelEncoder


In [22]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [23]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [24]:
from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


ValueError: zero-size array to reduction operation maximum which has no identity

In [ ]:
input_shape = (48, 48, 1)
num_classes = 7


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

model = Sequential()

# Convolutional Layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())

# Fully connected Layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

# Output layer
model.add(Dense(num_classes, activation='softmax'))


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(x=x_train, y=y_train, batch_size=120, epochs=10, validation_data=(x_test, y_test))


In [ ]:
model.fit(x=x_train, y=y_train, batch_size=120, epochs=10, validation_data=(x_test, y_test))


In [ ]:
model.fit(x=x_train, y=y_train, batch_size=120, epochs=10, validation_data=(x_test, y_test))


In [ ]:
model_json = model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

In [ ]:
from keras.models import model_from_json

In [ ]:
json_file = open("/content/drive/MyDrive/ml_sample/facialexpression.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("/content/drive/MyDrive/ml_sample/facialexpression.h5")

In [ ]:
label = ['angry','disgust','fear','happy','neutral','sad','surprise']


In [ ]:
def ef(image):
    img = load_img(image,grayscale =  True )
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0


In [ ]:
image = '/content/yt/images/train/fear/10010.jpg'
print("original image is of fear")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
image = '/content/yt/images/train/fear/10010.jpg'
print("original image is of fear")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')

In [ ]:
image = '/content/yt/images/train/angry/0.jpg'
print("original image is of fear")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')

In [ ]:
!pip install opencv-python


In [ ]:
!pip install opencv-python-headless
!pip install requests
import cv2
import requests
from PIL import Image
from io import BytesIO
image_link = "https://t4.ftcdn.net/jpg/00/56/93/53/360_F_56935312_NiqxkRKOdGSJd86Tc2uLycL9fkUsIlRW.jpg"
response = requests.get(image_link)
image = Image.open(BytesIO(response.content))
cv_image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)


In [ ]:
import cv2
import requests
from keras.models import model_from_json
import numpy as np
from PIL import Image
from io import BytesIO
from google.colab.patches import cv2_imshow

# Load the model architecture from JSON file
json_file = open("/content/drive/MyDrive/ml_sample/facialexpression.json", "r")
model_json = json_file.read()
json_file.close()

# Load the model weights
model = model_from_json(model_json)
model.load_weights("/content/drive/MyDrive/ml_sample/facialexpression.h5")

# Haar cascade file for face detection
haar_file = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(haar_file)

def extract_features(image):
    feature = np.array(image)
    feature = feature.reshape(1, 48, 48, 1)
    return feature / 255.0

# Provide the image link
image_link = "https://img.freepik.com/free-photo/young-bearded-man-with-white-t-shirt_273609-7200.jpg?size=626&ext=jpg&ga=GA1.1.1395880969.1709856000&semt=ais"

# Download the image from the link
response = requests.get(image_link)
image = Image.open(BytesIO(response.content))

# Convert PIL image to OpenCV format
image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Detect faces in the image
faces = face_cascade.detectMultiScale(gray, 1.3, 5)

labels = {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}

# Process each face detected in the image
for (p, q, r, s) in faces:
    face = gray[q:q+s, p:p+r]
    cv2.rectangle(image, (p, q), (p+r, q+s), (255, 0, 0), 2)
    face = cv2.resize(face, (48, 48))
    img = extract_features(face)
    pred = model.predict(img)
    prediction_label = labels[pred.argmax()]
    cv2.putText(image, '%s' % (prediction_label), (p-10, q-10), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, (0, 0, 255))

# Display the processed image
cv2_imshow(image)


In [ ]:
import cv2

# Open the default webcam (index 0)
webcam = cv2.VideoCapture(0)

# Check if the webcam is opened successfully
if not webcam.isOpened():
    print("Error: Failed to open webcam.")
    exit()

# Main loop for video capture and display
while True:
    # Capture frame-by-frame
    ret, frame = webcam.read()

    # Check if the frame is retrieved successfully
    if not ret:
        print("Error: Failed to retrieve frame from webcam.")
        break

    # Display the captured frame
    cv2.imshow("Webcam", frame)

    # Wait for 'Esc' key press to exit
    key = cv2.waitKey(1)
    if key == 27:  # 'Esc' key
        break

# Release the webcam and close OpenCV windows
webcam.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
from keras.models import model_from_json
import numpy as np

# Load model from JSON and weights from H5 file
json_file = open("/content/drive/MyDrive/ml_sample/facialexpression.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("/content/drive/MyDrive/ml_sample/facialexpression.h5")

# Load Haar cascade for face detection
haar_file = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(haar_file)

# Function to extract features from image
def extract_features(image):
    feature = np.array(image)
    feature = feature.reshape(1, 48, 48, 1)
    return feature / 255.0

# Webcam setup
webcam = cv2.VideoCapture(0)
webcam = cv2.VideoCapture(0)
webcam = cv2.VideoCapture(0)
webcam = cv2.VideoCapture(0)

# Emotion labels
labels = {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}

# Main loop for video capture and emotion recognition
while True:
    ret, frame = webcam.read()
    ret, frame = webcam.read()
    ret, frame = webcam.read()

    if not ret:  # Check if frame retrieval was successful
        print("Error: Failed to retrieve frame from webcam.")
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Perform face detection
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    try:
        # Process each detected face
        for (x, y, w, h) in faces:
            face_img = gray[y:y+h, x:x+w]  # Extract face region
            face_img = cv2.resize(face_img, (48, 48))  # Resize to model input size
            features = extract_features(face_img)  # Extract features
            pred = model.predict(features)  # Perform prediction
            prediction_label = labels[pred.argmax()]  # Get predicted label
            cv2.putText(frame, prediction_label, (x-10, y-10), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, (0, 0, 255))
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)  # Draw rectangle around face

        # Display the processed frame
        cv2.imshow("Emotion Recognition", frame)

        # Check for 'Esc' key press to exit the loop
        key = cv2.waitKey(1)
        if key == 27:  # 'Esc' key
            break
    except cv2.error:
        pass

# Release the webcam and close all OpenCV windows
webcam.release()
cv2.destroyAllWindows()


In [ ]:
!pip install opencv-python-headless
!pip install mtcnn

In [ ]:
import cv2
cap = cv2.VideoCapture(1)
print(cap.isOpened())
cap.release()


In [ ]:
import cv2
from IPython.display import display, Image

# Open the default camera (index 0)
cap = cv2.VideoCapture(0)

# Check if the camera is opened successfully
if not cap.isOpened():
    print("Error: Could not open camera.")
else:
    try:
        # Capture a single frame from the camera
        ret, frame = cap.read()

        # Check if the frame is captured successfully
        if ret:
            # Display the captured frame in the notebook
            display(Image(data=cv2.imencode('.jpg', cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))[1].tobytes(), format='png'))

            # Save the captured frame to a file
            cv2.imwrite('captured_frame.jpg', frame)
            print("Captured frame saved as 'captured_frame.jpg'.")

            # Read and display the saved frame
            saved_frame = cv2.imread('captured_frame.jpg')
            display(Image(data=cv2.imencode('.jpg', cv2.cvtColor(saved_frame, cv2.COLOR_BGR2RGB))[1].tobytes(), format='png'))

        else:
            print("Error: Could not capture frame.")

    except Exception as e:
        print(f"An error occurred: {e}")

    finally:
        # Release the camera
        cap.release()
